Pontificia Universidad Católica de Chile <br>
Departamento de Ciencia de la Computación <br>
IIC2433 - Minería de Datos <br>
Primer Semestre 2025 <br>

<h1><center>Tarea 3</center></h1>
<h1><center>Vicente Lavagnino</center></h1>


Profesor: Marcelo Mendoza<br>
Fecha de entrega: 07 de Mayo del 2025



## Indicaciones

Se debe entregar **SOLO** el archivo .ipynb en el buzón respectivo en canvas.

**IMPORTANTE**:
- Se asignará puntaje por el código implementado y los comentarios asociados a resultados.
- El notebook debe tener todas las celdas de código ejecutadas.
- El carácter de esta tarea es INDIVIDUAL.


**NOMBRE ESTUDIANTE:**

## Librerías

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from collections import Counter

from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, MultiLabelBinarizer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import classification_report, f1_score, confusion_matrix, ConfusionMatrixDisplay

from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

##Importa acá las librerias que vayas a utilizar

## Introducción

El conjunto de datos cuenta con un total de 785 columnas. La columna "label" indica la clase a la que pertenece cada registro, es decir, el tipo de prenda representada en la imagen. A continuación, se detallan las clases asociadas a esta columna y su correspondencia numérica:

0: T-shirt/top

1: Trouser

2: Pullover

3: Dress

4: Coat

5: Sandal

6: Shirt

7: Sneaker

8: Bag

9: Ankle boot

Por otra parte, las columnas cuyo nombre comienza con "pixel" representan las características de cada imagen. En total, hay 784 columnas de este tipo, las cuales representan imágenes de tamaño 28 x 28 píxeles.


## Parte 1: Carga y Preprocesamiento de Datos (8 puntos)

En esta primera etapa, se debe cargar el conjunto de datos y luego analizar el comportamiento de algunas variables de interés. Posteriormente, se deberá llevar a cabo el proceso de normalización de los datos.



## 1.1 Carga de Datos (3 puntos)

Cargue el conjunto de datos utilizando la función read_csv de la biblioteca pandas y asígnelo a la variable df.
¿Cuántos registros (artículos) contiene el dataset?

Luego, grafique la cantidad de productos por clase utilizando la columna label.
¿El conjunto de datos está balanceado en cuanto a la distribución de clases?

RESPUESTA

**Observación:**

Utilice la siguiente celda de código para graficar las imágenes.

In [ ]:
#Se separa el dataset en label (y) y variables explicativas (X).
y = df.label.copy()
X = df.drop("label",axis=1)
# Graficar ejemplos
fig, axs = plt.subplots(nrows=10, ncols=10, figsize=(10, 10))
for idx, ax in enumerate(axs.ravel()):
    ax.imshow(X.to_numpy()[idx].reshape((28, 28)), cmap=plt.cm.copper) #notar que el dataframe X se lee como un arreglo de numpy para aplicar reshape y ajustar la dimensión por imagen
    ax.set_title(f"Label = {y.iloc[idx]}", fontsize=7, pad=3)
    ax.axis("off")
_ = fig.suptitle("Plot dataset", fontsize=12)

## 1.2 Normalizar datos (2 puntos)

Realice el proceso de normalización sobre la matriz de variables explicativas utilizando el método MinMaxScaler.

RESPUESTA

## 1.3 Reducción de dimensionalidad (3 puntos)

Aplique el método PCA sobre el dataset normalizado para reducir su dimensionalidad, asegurando que se retenga el 90% de la varianza.
¿Qué número de componentes se requiere?

RESPUESTA:

#Parte 2: Clasificador KNN (20 puntos)

**En esta sección, deberá trabajar con el conjunto de datos normalizado obtenido en la pregunta 1.3, al cual se le ha aplicado el método PCA para reducir su dimensionalidad.**

##2.1 Visualización PCA (3 puntos)

Utilizando los primeros dos componentes principales de PCA, cree un gráfico para visualizar la distribución de clases del conjunto de datos.

¿Se observa algún patrón en la distribución?

**Observación:**

Cada punto del gráfico debe tener un color de acuerdo a su label.



RESPUESTA:

## 2.2 Visualización con TSNE (3 puntos)

Ajuste un modelo de T-SNE utilizando 2 componentes. Justifique la elección de los hiperparámetros utilizados.

Construya un gráfico para visualizar los resultados del modelo.

¿Reconoce algún patrón en los datos? Compare los resultados obtenidos con el gráfico generado en el punto 2.1.

**Observación:**

Cada punto del gráfico debe tener un color de acuerdo a su label (variable y)

RESPUESTA:

##2.3 Crear conjuntos train y test (2 puntos)

Cree los conjuntos train y test, a partir del conjunto de datos obtenido en 1.3.Considere un conjuto de test equivalente al 20%.




RESPUESTA

## 2.4 Entrenar clasificador KNN (6 puntos)

Cree clasificadores utilizando KNeighborsClassifier. Para seleccionar el número de vecinos, ajuste el hiperparámetro en el rango de 3 a 15, tomando la decisión en función de la métrica F1-score (utilice el parámetro average="macro").

Luego, para el mejor clasificador, muestre las métricas de precisión, recall y F1-score por clase. Analice los resultados obtenidos.

**Observación:**

Para la métrica F1-score puede utilizar la siguiente función:

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html


RESPUESTA:

##2.5 Evaluación manual de casos (6 puntos)

Para profundizar en el análisis, construya una matriz de confusión que permita identificar en qué clases se producen las principales inconsistencias. Analice los resultados obtenidos.

Luego, considere el label que obtuvo la peor métrica en F1-score. Evalúe 20 casos en los que la etiqueta observada no coincida con la predicción. Presente un gráfico con los elementos considerados (como se visualiza en 1.1).

¿Puede reconocer algún patrón?


**Observación:**

Debe trabajar con el conjunto de test.

RESPUESTA:

#Parte 3: Creación de datos sintéticos (20 puntos)

En esta sección debe trabajar con los **conjuntos de train** creados en el punto 2.3. Es decir, **X_train** e **y_train**

## 3.1 Ajustar modelo GMM (6 puntos)

Determine el número óptimo de componentes para el modelo Gaussian Mixture Model utilizando la métrica AIC. Considere valores en el intervalo de 1 a 100, con un paso de 5 entre cada valor.

Luego, grafique la relación entre el número de componentes y el valor de la métrica AIC.

**Observación:**

Este paso puede demorar cerca de 30 minutos.

## 3.2 Crear datos sintéticos (8 puntos)

Cree una muestra de 30 elementos sintéticos utilizando el mejor modelo ajustado en el punto anterior.

Luego, grafique los datos generados y comente sobre su calidad.



RESPUESTA:

## 3.3 Evaluar calidad (6 puntos)

Aplique el clasificador creado en el punto 2.4 sobre los ejemplos sintéticos generados.

¿El clasificador es capaz de detectar correctamente las imágenes?

Analice los resultados obtenidos.



RESPUESTA:

#Parte 4: Reglas de asociación (12 puntos)

En esta sección se trabajará con un dataset que simula el comportamiento de usuarios en una plataforma de streaming 🎥, registrando las series de TV que han visualizado. Cada fila del archivo representa a un usuario distinto (un total de 100 usuarios) e incluye una lista de las series que ha visto un usuario durante un período determinado.

Las series están separadas por comas (",").

El dataset está disponible en Canvas.

## 4.1 Carga de datos y One hot encoding (3 puntos)

Cargue el dataset utilizando la función read_csv de pandas. ¿Cuál o cuáles son las series más frecuentes dentro del dataset?

Luego, utilice TransactionEncoder() para codificar las transacciones en vectores.

RESPUESTA

## 4.2 Matriz de co-ocurrencia (3 puntos)



Genera la matriz de co-ocurrencia para observar con qué frecuencia las series de TV aparecen juntas.

¿Qué pares de series parecen estar fuertemente relacionadas, sin considerar la diagonal de la matriz?

¿Qué hipótesis puedes plantear sobre por qué estas series se ven juntas?



RESPUESTA

## 4.3 Algoritmo Apriori (3 puntos)

Aplica el algoritmo Apriori con min_support = 0.2 y analiza los resultados obtenidos.

A continuación, genera las reglas de asociación utilizando association_rules, con un umbral min_threshold = 0.6, y comenta los resultados obtenidos.

¿Cómo podrían estas reglas ser utilizadas para hacer recomendaciones personalizadas a los usuarios?

RESPUESTA

## 4.4 Aplicación del Algoritmo Apriori Excluyendo 'Better Call Saul' y 'Breaking Bad' (3 puntos)

Vuelve a aplicar el algoritmo Apriori utilizando la matriz de vectores normalizados (one-hot encoding), excluyendo las series 'Better Call Saul' y 'Breaking Bad'.

Prueba el algoritmo variando el hiperparámetro min_support en los valores [0.05, 0.1, 0.2]. Luego, para el min_threshold, prueba valores en el rango [0.1, 0.3, 0.6].

¿Es posible generar reglas utilizando los mismos valores de hiperparámetro utilizados en el punto 4.3? Si no se generan reglas, ajusta los valores dentro de los rangos de parámetros posibles para cada uno y comenta los resultados.

Finalmente, explica qué significan los hiperparámetros min_support y min_threshold en el contexto del algoritmo Apriori.


RESPUESTA